In [296]:
import geopandas as gpd
import pandas as pd
import numpy as np
import urllib
import requests
import os
import io
import json
import pylab as pl
import shapely
import choroplethNYC as cp
#from fiona.crs import from_epsg

%pylab inline

import scipy as sp
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


# Download Parks Inspection Data

In [270]:
#Loading csv of park inspections into dataframe 
inspection = pd.read_csv("https://data.cityofnewyork.us/api/views/yg3y-7juh/rows.csv?accessType=DOWNLOAD")

In [271]:
#looking through the Overall Celanliness column 
parksinspection = inspection

# Convert the "Overall condition" column to a numerical value.

In [272]:
parksinspection['Overall Condition'] = parksinspection['Overall Condition'].map({'A': 1, 'U': 0, 'N': np.nan})

In [273]:
parksinspection['Overall Condition'].dtype

dtype('float64')

# Group the dataframe by park park id and get the mean condition for each park.

In [274]:
parksinspection.head()

Prop ID AMPSDistrict  Inspection ID  Season  Round                    Date  \
0   Q066A            1          68214  Summer      1  05/24/2010 12:00:00 AM   
1   QZ284            4          68220  Summer      1  05/24/2010 12:00:00 AM   
2   Q121A           12          68227  Summer      1  05/24/2010 12:00:00 AM   
3    Q301           12          68228  Summer      1  05/24/2010 12:00:00 AM   
4    M196            3          68234  Summer      1  05/25/2010 12:00:00 AM   

  BeginInspection EndInspection  Inspection Year  inspector  inspector2  \
0        12:30 PM      12:46 PM             2010          3         NaN   
1        11:45 AM      11:55 AM             2010         15         NaN   
2        11:05 AM      11:35 AM             2010         18         NaN   
3        10:20 AM      10:54 AM             2010         18         NaN   
4        09:25 AM      09:37 AM             2010          6         NaN   

   Overall Condition Cleanliness Safety Condition Structural Condition  \
0                1.0           A              NaN                  NaN   
1                1.0           A              NaN                  NaN   
2                1.0           A              NaN                  NaN   
3                0.0           U              NaN                  NaN   
4                1.0           A              NaN                  NaN   

   VisitorCount                              Closed?  \
0           NaN                  Under Construction.   
1           NaN                                  NaN   
2           NaN                                  NaN   
3           NaN                                  NaN   
4           NaN  Partial Constr./Rest of Site Rated.   

                                            Comments InspectionType  \
0  Site closed for bridge maintenance, exterior r...            PIP   
1                                                NaN            PIP   
2                                                NaN            PIP   
3                                                NaN            PIP   
4             Most of park closed for school repair.            PIP   

         inspAddedDate  
0  2010-05-24 14:40:00  
1  2010-05-24 14:56:00  
2  2010-05-24 15:23:00  
3  2010-05-24 15:37:00  
4  2010-05-25 12:07:00

In [275]:
parksinspection['Overall Condition'] = parksinspection['Overall Condition'].dropna(axis=0)

In [276]:
parksinspection.head()

Prop ID AMPSDistrict  Inspection ID  Season  Round                    Date  \
0   Q066A            1          68214  Summer      1  05/24/2010 12:00:00 AM   
1   QZ284            4          68220  Summer      1  05/24/2010 12:00:00 AM   
2   Q121A           12          68227  Summer      1  05/24/2010 12:00:00 AM   
3    Q301           12          68228  Summer      1  05/24/2010 12:00:00 AM   
4    M196            3          68234  Summer      1  05/25/2010 12:00:00 AM   

  BeginInspection EndInspection  Inspection Year  inspector  inspector2  \
0        12:30 PM      12:46 PM             2010          3         NaN   
1        11:45 AM      11:55 AM             2010         15         NaN   
2        11:05 AM      11:35 AM             2010         18         NaN   
3        10:20 AM      10:54 AM             2010         18         NaN   
4        09:25 AM      09:37 AM             2010          6         NaN   

   Overall Condition Cleanliness Safety Condition Structural Condition  \
0                1.0           A              NaN                  NaN   
1                1.0           A              NaN                  NaN   
2                1.0           A              NaN                  NaN   
3                0.0           U              NaN                  NaN   
4                1.0           A              NaN                  NaN   

   VisitorCount                              Closed?  \
0           NaN                  Under Construction.   
1           NaN                                  NaN   
2           NaN                                  NaN   
3           NaN                                  NaN   
4           NaN  Partial Constr./Rest of Site Rated.   

                                            Comments InspectionType  \
0  Site closed for bridge maintenance, exterior r...            PIP   
1                                                NaN            PIP   
2                                                NaN            PIP   
3                                                NaN            PIP   
4             Most of park closed for school repair.            PIP   

         inspAddedDate  
0  2010-05-24 14:40:00  
1  2010-05-24 14:56:00  
2  2010-05-24 15:23:00  
3  2010-05-24 15:37:00  
4  2010-05-25 12:07:00

In [277]:
parksinspection = parksinspection.groupby('Prop ID', as_index=False)['Overall Condition'].mean()

In [278]:
parksinspection.head()

Prop ID  Overall Condition
0    B001           0.897436
1    B002           0.761905
2    B003           1.000000
3    B006           1.000000
4    B007           0.314286

# 2. Download parks shapefile for the NYC Open Data data.

In [279]:
#checking environmental variable is set properly
os.getenv("PUIDATA")

'/nfshome/ram844/PUIdata'

In [280]:
#downloading the zipped package containing the parks inspection shapefile to my PUIdata folder
url = "https://data.cityofnewyork.us/api/geospatial/k2ya-ucmv?method=export&format=Shapefile"
urllib.request.urlretrieve(url, "parkfile.gz")
!unzip -d $PUIDATA parkfile.gz

Archive:  parkfile.gz
  inflating: /nfshome/ram844/PUIdata/geo_export_13c21d79-254c-42b3-88c7-327a22ac7bee.dbf  
  inflating: /nfshome/ram844/PUIdata/geo_export_13c21d79-254c-42b3-88c7-327a22ac7bee.shp  
  inflating: /nfshome/ram844/PUIdata/geo_export_13c21d79-254c-42b3-88c7-327a22ac7bee.shx  
  inflating: /nfshome/ram844/PUIdata/geo_export_13c21d79-254c-42b3-88c7-327a22ac7bee.prj  


In [281]:
#putting parks inspection shapefile into geopandas dataframe
geoparksinspection = gpd.GeoDataFrame.from_file(os.getenv("PUIDATA") + "/" + "geo_export_084a16e9-3dbb-4516-a6fe-91c6184a1641.shp")
geoparksinspection

acquisitio    acres                    address borough class  \
0     19440801000000.00000    0.991                       None       M  PLGD   
1                     None    0.030                       None       Q  PARK   
2     19360824000000.00000    0.911           2300 Ryer Avenue       X  PARK   
3     19520206000000.00000    0.189          349 RODNEY STREET       B  PARK   
4     19240403000000.00000    0.554              124 11 STREET       B  PARK   
5     19380606000000.00000    0.036                       None       B  PARK   
6     20030304000000.00000    0.124                       None       R  PARK   
7     19291120000000.00000  920.426                       None       R  PARK   
8     19550127000000.00000    1.210        9920 SEAVIEW AVENUE       B  PARK   
9     19340626000000.00000    0.567    358 SCHERMERHORN STREET       B  PARK   
10    19350809000000.00000    1.240                       None       B  PARK   
11    19420209000000.00000    1.171                       None       B  PARK   
12    19710114000000.00000    3.259       6900 HYLAN BOULEVARD       R  PARK   
13    19060110000000.00000    0.340                       None       M  PARK   
14    19940325000000.00000  178.470                       None       R  PARK   
15    20160209000000.00000    0.136                       None       X  PARK   
16    19971219000000.00000    0.118        128 WEST 139 STREET       M  PLGD   
17    19400606000000.00000    0.197         4814 KINGS HIGHWAY       B  PARK   
18    19460131000000.00000    0.461                       None       X  PARK   
19                    None    0.028                       None       Q  PARK   
20    19550615000000.00000    1.030                       None       Q  PARK   
21    19630507000000.00000    0.082                       None       Q  PARK   
22    19120102000000.00000    0.229    902 CROSS BAY BOULEVARD       Q  PARK   
23    19661014000000.00000    0.006                       None       X  PARK   
24    19061010000000.00000    3.900                  294 10 Av       M  PARK   
25    19350313000000.00000    0.034                       None       M  PARK   
26    19570328000000.00000    0.031                       None       X  PARK   
27    19611026000000.00000    1.616        10002 GLENWOOD ROAD       B  PARK   
28    19481118000000.00000    1.025                       None       X  PARK   
29    19370507000000.00000    0.460                       None       Q  PARK   
...                    ...      ...                        ...     ...   ...   
1977  20021120000000.00000    0.219              750 Howard Av       B  PARK   
1978  20060119000000.00000    0.115       2502 DAVIDSON AVENUE       X  PARK   
1979  19961231000000.00000    0.401                       None       X  PARK   
1980  20040419000000.00000    0.084        323 WEST 126 STREET       M  PARK   
1981  20060814000000.00000    0.160        1311 BRISTOW STREET       X  PARK   
1982  19970724000000.00000    0.383        342 PLEASANT AVENUE       M  PARK   
1983  20031117000000.00000    0.174  379 - 383 EAST 159 STREET       X  PARK   
1984  20130325000000.00000    0.039        340 TOMPKINS AVENUE       B  PARK   
1985  20090128000000.00000   42.583                       None       B  PARK   
1986  19600728000000.00000    0.464            81-02 24 AVENUE       Q  PARK   
1987  20021120000000.00000    0.033         213 MADISON STREET       B  PARK   
1988  20131101000000.00000   22.600          52-10 CENTER BLVD       Q  PARK   
1989  20021120000000.00000    0.121                       None       Q  PARK   
1990  19990712000000.00000    0.206              1464 BROADWAY       B  PARK   
1991  20121120000000.00000    0.161        532 GLENMORE AVENUE       B  PARK   
1992  20021120000000.00000    0.249          218 EAST 2 STREET       M  PARK   
1993  19340712000000.00000    1.352                       None       R  PARK   
1994  19961231000000.00000    0.286                       None       X  PARK   
1995  2002112000

# Extract the Brooklyn park properties from this file 

In [282]:
bkParks = geoparksinspection[geoparksinspection["borough"] == "B"] # reduces to only Brooklyn portion of the data frame
bkParks

acquisitio   acres                      address borough class  \
3     19520206000000.00000   0.189            349 RODNEY STREET       B  PARK   
4     19240403000000.00000   0.554                124 11 STREET       B  PARK   
5     19380606000000.00000   0.036                         None       B  PARK   
8     19550127000000.00000   1.210          9920 SEAVIEW AVENUE       B  PARK   
9     19340626000000.00000   0.567      358 SCHERMERHORN STREET       B  PARK   
10    19350809000000.00000   1.240                         None       B  PARK   
11    19420209000000.00000   1.171                         None       B  PARK   
17    19400606000000.00000   0.197           4814 KINGS HIGHWAY       B  PARK   
27    19611026000000.00000   1.616          10002 GLENWOOD ROAD       B  PARK   
32    19500309000000.00000   1.225        126 BRIGHTWATER COURT       B  PARK   
35    18950101000000.00000  16.800                         None       B  PARK   
36    20021120000000.00000   0.190           93 NEW LOTS AVENUE       B  PARK   
38    19460501000000.00000   1.093                         None       B  PARK   
44    19520206000000.00000   0.104             153 MARCY AVENUE       B  PARK   
48                    None   6.020              8301 SHORE ROAD       B  PARK   
49    19810405000000.00000   1.161      343 FLATBUSH AVENUE EXT       B  PARK   
50    19940303000000.00000  64.665                         None       B  PARK   
58    19520206000000.00000   0.357          243 DIVISION AVENUE       B  PARK   
59    19361208000000.00000   0.327           106 SOUTH 3 STREET       B  PARK   
61    20060814000000.00000   0.321           590 SCHENCK AVENUE       B  PARK   
64    19540527000000.00000   1.230           525 BEDFORD AVENUE       B  PARK   
76    19581218000000.00000   0.116              1 EAST 8 STREET       B  PARK   
78    19570828000000.00000   1.260            175 ELLERY STREET       B  PARK   
79    19610609000000.00000   1.061           125 CENTRAL AVENUE       B  PARK   
86    19571024000000.00000   0.790         1060 CLARKSON AVENUE       B  PARK   
91    20130423000000.00000   1.351                         None       B  PARK   
97    19231025000000.00000   0.057                         None       B  PARK   
98    19470514000000.00000   0.026                         None       B  PARK   
99    19460501000000.00000   1.500              233 GOLD STREET       B  PARK   
100   19460514000000.00000   0.762            178 BAYARD STREET       B  PARK   
...                    ...     ...                          ...     ...   ...   
1943  20160119000000.00000   0.060          615 SARATOGA AVENUE       B  PARK   
1944  20140723000000.00000   0.097           1278 MYRTLE AVENUE       B  PARK   
1945  19970422000000.00000   0.185  385 - 389 KOSCIUSZKO STREET       B  PARK   
1946  20021120000000.00000   0.090   433 - 435 CLEVELAND STREET       B  PARK   
1947  20020220000000.00000   0.058            48 LEFFERTS PLACE       B  PARK   
1948  20021120000000.00000   0.064          147 TOMPKINS AVENUE       B  PARK   
1953  20021120000000.00000   0.049                         None       B  PARK   
1955  20121128000000.00000   0.052            100 QUINCY STREET       B  PARK   
1956  20121120000000.00000   0.160             41 COOPER STREET       B  PARK   
1959  20021120000000.00000   0.041           601 WARWICK STREET       B  PARK   
1960  20160209000000.00000   0.069        87 SCHENECTADY AVENUE       B  PARK   
1964  20021120000000.00000   0.114          555-557 Shepherd Av       B  PARK   
1966  20031117000000.00000   0.092           398-400 Montauk Av       B  PARK   
1967  20060309000000.00000   0.115        335 VAN SICLEN AVENUE       B  PARK   
1968  20021120000000.00000   0.206             790 BLAKE AVENUE       B  PARK   
1971  20021120000000.00000   0.058              1711 PARK PLACE       B  PARK   
1972  20021120000000.00000   0.119           444-448 Warwick St       B  PARK   
1974  19990712000000.00000   0.082             

In [283]:
bkParks.department(['B-', np.nan]).str.replace('B0', regex=True)

TypeError: 'Series' object is not callable

In [284]:
bkParks.shape

(608, 36)

In [285]:
#ax = bkParks.plot(column="condition", k=3, cmap="RdBu", legend=True, figsize=(10,10) )
#ax.set_ylabel("latitude", fontsize=20)
#ax.set_xlabel("longitude", fontsize=20);

# 3 Download unemployment rate information from ACS 2015 (5 year survey) for Brooklyn at the PUMA geographical aggregation level.

In [ ]:
#url = "https://api.census.gov/data/2015/acs/acs1/variables.json"
#resp = requests.request('GET', url)
#aff1y = json.loads(resp.text)

In [286]:
#acs15_final.csv collected using wget
pumaUnemployment = pd.read_csv(os.getenv("PUIDATA") + "/acs15_final.csv")

In [287]:
pumaUnemployment.head()

Unnamed: 0  unemployedF  total  public use microdata area
0           0     0.118098   1304                        100
1           1     0.120651   1782                        200
2           2     0.086304   1066                        300
3           3     0.117403    724                        401
4           4     0.083603   2476                        402

# 4. Obtain the NYC shapefile for PUMA

In [288]:
#downloading the zipped package containing the NYC shapefile for PUMA and unzipping to my PUIdata folder
url2 = "https://data.cityofnewyork.us/api/geospatial/cwiz-gcty?method=export&format=Shapefile"
urllib.request.urlretrieve(url2, "pumafile.gz")
!unzip -d $PUIDATA pumafile.gz


Archive:  pumafile.gz
  inflating: /nfshome/ram844/PUIdata/geo_export_fbcad0c5-6f8d-4459-a361-9d2b942c5358.dbf  
  inflating: /nfshome/ram844/PUIdata/geo_export_fbcad0c5-6f8d-4459-a361-9d2b942c5358.shp  
  inflating: /nfshome/ram844/PUIdata/geo_export_fbcad0c5-6f8d-4459-a361-9d2b942c5358.shx  
  inflating: /nfshome/ram844/PUIdata/geo_export_fbcad0c5-6f8d-4459-a361-9d2b942c5358.prj  


In [289]:
#putting puma shapefile into geopandas dataframe

pumashp = gpd.GeoDataFrame.from_file(os.getenv("PUIDATA") + "/" + 
                                     "geo_export_88087a8c-7575-4cbe-bb64-41eb51e0f4a0.shp")

In [290]:
pumashp.head()

puma    shape_area     shape_leng  \
0  3701  9.792852e+07   53227.144461   
1  3702  1.889860e+08  106050.002302   
2  3703  2.670137e+08  304071.257466   
3  3704  1.062129e+08   47970.901277   
4  4015  8.105438e+07   50007.415493   

                                            geometry  
0  POLYGON ((-73.89641133483133 40.90450452082026...  
1  POLYGON ((-73.8442314689986 40.86942035096838,...  
2  (POLYGON ((-73.78833349834532 40.8346671297593...  
3  POLYGON ((-73.84792614069238 40.8713422330779,...  
4  POLYGON ((-73.95374853778468 40.63858633758547...

In [ ]:
# combine lat and lon to one column
pumashp['lonlat'] = list(zip(linkNYC.longitude, linkNYC.latitude))
# Create Point Geometry for based on lonlat column
linkNYC['geometry'] = linkNYC[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

In [299]:
pumashp.plot

<bound method GeoDataFrame.plot of     puma    shape_area     shape_leng  \
0   3701  9.792852e+07   53227.144461   
1   3702  1.889860e+08  106050.002302   
2   3703  2.670137e+08  304071.257466   
3   3704  1.062129e+08   47970.901277   
4   4015  8.105438e+07   50007.415493   
5   4016  1.207135e+08  109477.751817   
6   3705  1.224951e+08   68657.316149   
7   3706  4.388687e+07   51799.404183   
8   3707  4.228113e+07   37347.925798   
9   4006  6.184965e+07   42555.996557   
10  3708  5.589695e+07   34852.437966   
11  3709  1.241177e+08   73288.964465   
12  3710  1.377966e+08   91086.012338   
13  3801  8.124958e+07   64103.138480   
14  4011  4.762886e+07   34965.040151   
15  4012  1.123044e+08   97261.951304   
16  3802  4.689908e+07   37925.566208   
17  4014  8.768042e+07   51089.073159   
18  3803  3.984279e+07   38064.749400   
19  3804  6.461212e+07   62599.500502   
20  3805  5.516881e+07   53561.096534   
21  3806  8.849786e+07   47538.426032   
22  3807  8.560261e+07

# 5 Download the file assigning each park to a Puma

In [301]:
parkByPuma = pd.read_csv("https://serv.cusp.nyu.edu/~fbianco/PUIdata/parkToPuma.csv")

In [302]:
parkByPuma.head()

puma  condition  gispropnum
0  4001   0.633277          52
1  4002   0.456086          17
2  4003   0.384951          23
3  4004   0.648573          49
4  4005   0.718519          29

# 6 Data Aggregation

merge the dataframes and geodataframes (pumas, park inspections, unemployment), group them so that each PUMA has

the unemployed fraction
the average park condition
the number of parks per PUMA
the shape of the area (PUMA)
Your deliverable for this task is a dataframe like the one below

You can find the solution here - if you use this solution you will not get the 10 points for this deliverable (but you can go back later to this! and if you do get the solution later you will get the points!) You will stil have to secure the PUMA shape file if you want to make the plots and merge it with this file



In [306]:
bkmerged = pumashp.merge(parkByPuma, left_on='puma', right_on='puma', how='outer')

In [307]:
bkmerged.head()

puma    shape_area     shape_leng  \
0  3701  9.792852e+07   53227.144461   
1  3702  1.889860e+08  106050.002302   
2  3703  2.670137e+08  304071.257466   
3  3704  1.062129e+08   47970.901277   
4  4015  8.105438e+07   50007.415493   

                                            geometry  condition  gispropnum  
0  POLYGON ((-73.89641133483133 40.90450452082026...        NaN         NaN  
1  POLYGON ((-73.8442314689986 40.86942035096838,...        NaN         NaN  
2  (POLYGON ((-73.78833349834532 40.8346671297593...        NaN         NaN  
3  POLYGON ((-73.84792614069238 40.8713422330779,...        NaN         NaN  
4  POLYGON ((-73.95374853778468 40.63858633758547...        NaN         NaN